In [ ]:
from bs4 import BeautifulSoup   # The data scrapping library
import requests                 # To get the content of the web page
import re                       # Regular Expressison Library
import time                     # To introduce time delay
import pandas as pd             # Library to create Dataframe
from tqdm import tqdm 

In [3]:
# Lists to store players href data and names
player_names = []
player_hrefs = []

# Reading the player's href from the players.txt file
with open('.\\Data\\players.txt', 'r', encoding='utf8') as file_obj:
    for i in file_obj.readlines():
        player_names.append(i.strip().split('__')[0])
        player_hrefs.append(i.strip().split('__')[1])

In [4]:
def create_table(soup, player_name):
    
    # Traversing the soup to get the the "Per Game" table
    per_game = soup.find(id = "div_per_game")
    table = per_game.tbody.find_all(class_ = 'full_table')

    # List to store the player data
    player_data = []

    # Extracting the data of player row wise from the table
    for row in table:
        year = [row.th.text]
        data = row.find_all('td')

        for i in data:
            year.append(i.text)

        player_data.append(year)

    # Getting the column header
    columns = per_game.thead.text.strip()
    columns = columns.split()

    # Inserting the data into a Dataframe
    df = pd.DataFrame(player_data, columns = columns)
    file_name = re.sub('\.\s|\.|\s|-', '_', player_name.strip().lower())

    # Storing the data as csv file
    df.to_csv('.\\Data\\Stats\\'+file_name+'.csv', index=False)

In [ ]:
pbar = tqdm(total=len(player_names))
base_url = 'https://www.basketball-reference.com'

for i in range(len(player_names)):
    # Getting the player page data
    response = requests.get(base_url+player_hrefs[i])
    
    # Creating Beautiful Soup object
    soup = BeautifulSoup(response.content, 'lxml')
    
    # Calling function to extract player data
    create_table(soup, player_names[i])
    
    # Due to rate limitation
    time.sleep(2)
    
    pbar.update(1)